In [85]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import time
import urllib.parse

In [86]:
# Sua chave de API Alpha Vantage
api_key = 'Y4E1GMWHA4EWZRHY'

# Lista de 10 símbolos de ações importantes
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'JPM', 'V', 'DIS', 'FB']

In [87]:
# Função para buscar dados de ações
def get_stock_data(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=60min&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if 'Time Series (60min)' in data:
        time_series = data['Time Series (60min)']
        df = pd.DataFrame(time_series).T
        df = df.reset_index().rename(columns={'index': 'timestamp'})
        df['symbol'] = symbol
        return df
    else:
        print(f"Erro ao buscar dados para {symbol}")
        return pd.DataFrame()

In [88]:
# Função para obter dados de várias ações e consolidar em um dataframe
def fetch_all_data(symbols):
    all_data = pd.DataFrame()
    for symbol in symbols:
        stock_data = get_stock_data(symbol)
        if not stock_data.empty:
            all_data = pd.concat([all_data, stock_data])
        time.sleep(12)  # Respeitar o limite de requisições da API
    return all_data

In [89]:
# Parâmetros de conexão
usuario = "root"
senha = "Are@0051"
host = "localhost"  
nome_do_banco = "shares"
senha_codificada = urllib.parse.quote_plus(senha)

conexao_string = f"mysql+pymysql://{usuario}:{senha_codificada}@{host}/{nome_do_banco}"

engine = create_engine(conexao_string)

In [90]:
# Função principal para atualizar o banco de dados
def update_stock_data():
    df = fetch_all_data(symbols)

    if not df.empty:
        # Remover índices antigos e formatar colunas
        df = df.reset_index(drop=True)
        df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'symbol']
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Inserir no MySQL com append para manter o histórico
        df.to_sql('stock_prices', con=engine, if_exists='append', index=False)
        print("Dados atualizados com sucesso!")
    else:
        print("Nenhum dado foi inserido.")

# Executar o script para coletar os dados e atualizar o banco de dados
update_stock_data()

Erro ao buscar dados para AAPL
Erro ao buscar dados para MSFT


KeyboardInterrupt: 